In [7]:
import h5py
import numpy as np
import pandas as pd
import os
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider

In [3]:
# --- 1. Load Data and Organize ---
data = defaultdict(lambda: defaultdict(list))
file_dir = "F:/Recovered/Research/BoninLab/PainModelingProject/ForCumin/0um_h5/test_subset"  # Replace with the actual path

for filename in os.listdir(file_dir):
    if filename.endswith('.h5') and 'df_f_traces' in filename:
        match = re.search(r'(Cap|CFA|SNI|Sham|Disinhib|OA|Paclitaxel)(\d+)_(\d+\.\d+\.\d+)_([a-z]+)(\d+)', filename)
        if match:
            model = match.group(1)
            side = 'ipsi' if 'ipsi' in filename else 'contra'
            try:
                with h5py.File(os.path.join(file_dir, filename), 'r') as f:
                    traces = np.array(f['data'])
                    data[model][side].append(traces)
            except Exception as e:
                print(f"Error reading {filename}: {e}")

print("Data loaded and organized.")

Error reading CFA6_2.4.20_contra3_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading CFA7_2.5.20_contra3_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading CFA7_2.5.20_ipsi1_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading CFA7_2.5.20_ipsi2_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading Disinhib3_7.29.20_ipsi2_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading Disinhib4_8.4.20_ipsi2_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading Disinhib7_1.27.21_contra2_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 'data' doesn't exist)"
Error reading Disinhib7_1.27.21_contra3_0um_cor_df_f_traces.h5: "Unable to synchronously open object (object 

In [9]:
# --- 2. Event Detection with Interactive Threshold ---
event_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

def interactive_event_detection(threshold):
    event_data.clear()  # Clear previous results
    for model, sides in data.items():
        for side, all_traces in sides.items():
            for slice_traces in all_traces:
                events_per_roi = [detect_events(trace, threshold=threshold) for trace in slice_traces]
                event_data[model][side].append(events_per_roi)
    print(f"Events detected with threshold: {threshold}")
    return event_data

@interact(threshold=FloatSlider(min=0.1, max=2.0, step=0.01, value=0.5, description='Event Threshold'))
def run_event_detection(threshold):
    global event_data
    event_data = interactive_event_detection(threshold)
    # Optionally, you could trigger the subsequent analyses here if you want to see results update with the threshold

# Placeholder for a message after interactive detection
print("Adjust the threshold slider above to detect events.")

def detect_events(trace, threshold=0.5):
    """Detects events based on a simple amplitude threshold."""
    return np.where(trace > threshold)[0]


interactive(children=(FloatSlider(value=0.5, description='Event Threshold', max=2.0, min=0.1, step=0.01), Outp…

Adjust the threshold slider above to detect events.


In [10]:
# --- 3. Co-occurrence Analysis ---
co_occurrence_results = defaultdict(lambda: defaultdict(list))

def calculate_co_occurrence(events1, events2, window=10):
    co_occurrences = 0
    for e1 in events1:
        for e2 in events2:
            if abs(e1 - e2) <= window:
                co_occurrences += 1
    return co_occurrences

def run_co_occurrence_analysis(event_data, window=10):
    co_occurrence_results.clear()
    for model, sides in event_data.items():
        for side, all_slices_events in sides.items():
            for slice_events in all_slices_events:
                num_rois = len(slice_events)
                co_occurrence_matrix = np.zeros((num_rois, num_rois))
                for i in range(num_rois):
                    for j in range(i + 1, num_rois):
                        co_occurrences = calculate_co_occurrence(slice_events[i], slice_events[j], window=window)
                        co_occurrence_matrix[i, j] = co_occurrences
                        co_occurrence_matrix[j, i] = co_occurrences
                co_occurrence_results[model][side].append(co_occurrence_matrix)
    print(f"Co-occurrence matrices calculated with window: {window}")
    return co_occurrence_results

@interact(window=IntSlider(min=1, max=50, step=1, value=10, description='Co-occurrence Window'))
def visualize_co_occurrence(window):
    if event_data:
        co_occurrence_results = run_co_occurrence_analysis(event_data, window)
        plot_connectivity_distributions(co_occurrence_results, f'Distribution of Co-occurrence (Pairwise, Window={window})')
    else:
        print("Please run event detection first.")

interactive(children=(IntSlider(value=10, description='Co-occurrence Window', max=50, min=1), Output()), _dom_…

In [14]:
# --- 4. Cross-Correlation Analysis ---
cross_correlation_results = defaultdict(lambda: defaultdict(list))

def binarize_trace(trace, threshold):
    return (trace > threshold).astype(int)

def calculate_peak_cross_correlation(trace1_bin, trace2_bin, max_lag=50):
    corr = np.correlate(trace1_bin, trace2_bin, mode='same')
    lags = np.arange(-len(trace1_bin) + 1, len(trace1_bin))
    zero_lag_index = np.where(lags == 0)[0][0]
    corr_without_zero = np.concatenate((corr[:zero_lag_index], corr[zero_lag_index+1:]))
    if corr_without_zero.size > 0:
        return np.max(np.abs(corr_without_zero)) / np.sqrt(np.sum(trace1_bin) * np.sum(trace2_bin)) if (np.sum(trace1_bin) > 0 and np.sum(trace2_bin) > 0) else 0
    else:
        return 0

def run_cross_correlation_analysis(data, threshold, max_lag=50):
    cross_correlation_results.clear()
    for model, sides in data.items():
        for side, all_traces in sides.items():
            for slice_traces in all_traces:
                num_rois = len(slice_traces)
                peak_corr_matrix = np.zeros((num_rois, num_rois))
                binarized_traces = [binarize_trace(trace, threshold) for trace in slice_traces]
                for i in range(num_rois):
                    for j in range(i + 1, num_rois):
                        peak_corr = calculate_peak_cross_correlation(binarized_traces[i], binarized_traces[j], max_lag=max_lag)
                        peak_corr_matrix[i, j] = peak_corr
                        peak_corr_matrix[j, i] = peak_corr
                cross_correlation_results[model][side].append(peak_corr_matrix)
    print(f"Peak cross-correlation matrices calculated with threshold: {threshold}, max_lag: {max_lag}")
    return cross_correlation_results

@interact(threshold=FloatSlider(min=0.1, max=2.0, step=0.1, value=0.5, description='Threshold for Binarization'),
          max_lag=IntSlider(min=10, max=200, step=10, value=50, description='Max Lag'))
def visualize_cross_correlation(threshold, max_lag):
    cross_correlation_results = run_cross_correlation_analysis(data, threshold, max_lag)
    plot_connectivity_distributions(cross_correlation_results, f'Distribution of Peak Cross-Correlation (Pairwise, Threshold={threshold}, Max Lag={max_lag})')

interactive(children=(FloatSlider(value=0.5, description='Threshold for Binarization', max=2.0, min=0.1), IntS…

In [15]:
# --- 5. Visualization Function ---
def plot_connectivity_distributions(results, title):
    data_to_plot = []
    for model, sides in results.items():
        for side, all_matrices in sides.items():
            for matrix in all_matrices:
                upper_triangle = matrix[np.triu_indices_from(matrix, k=1)]
                df = pd.DataFrame({'value': upper_triangle, 'model': model, 'side': side})
                data_to_plot.append(df)

    if data_to_plot:
        combined_df = pd.concat(data_to_plot)
        plt.figure(figsize=(12, 6))
        sns.boxplot(x='model', y='value', hue='side', data=combined_df)
        plt.title(title)
        plt.ylabel('Connectivity Strength')
        plt.xlabel('Pain Model')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    else:
        print("No data to plot.")

from ipywidgets import IntSlider

# Add an interactive slider for the co-occurrence window

print("Ready for interactive analysis and visualization!")
print("Adjust the sliders to explore different parameters.")

Ready for interactive analysis and visualization!
Adjust the sliders to explore different parameters.


In [ ]:
from scipy import stats
import pandas as pd
import statsmodels.api as sa
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def calculate_summary_metric(matrix, metric='mean'):
    """Calculates a summary metric from the connectivity matrix."""
    upper_triangle = matrix[np.triu_indices_from(matrix, k=1)]
    if metric == 'mean':
        return np.mean(upper_triangle)
    elif metric == 'max':
        return np.max(upper_triangle)
    # Add other metrics as needed

summary_results_cooc = defaultdict(lambda: defaultdict(list))
for model, sides in co_occurrence_results.items():
    for side, all_matrices in sides.items():
        for matrix in all_matrices:
            summary_results_cooc[model][side].append(calculate_summary_metric(matrix))

summary_results_corr = defaultdict(lambda: defaultdict(list))
for model, sides in cross_correlation_results.items():
    for side, all_matrices in sides.items():
        for matrix in all_matrices:
            summary_results_corr[model][side].append(calculate_summary_metric(matrix))

# Example: Comparing mean co-occurrence between Sham and CFA (ipsi)
sham_ipsi_cooc = summary_results_cooc['Sham']['ipsi']
cfa_ipsi_cooc = summary_results_cooc['CFA']['ipsi']
if sham_ipsi_cooc and cfa_ipsi_cooc:
    t_stat, p_val = stats.ttest_ind(sham_ipsi_cooc, cfa_ipsi_cooc)
    print(f"T-test (Sham-ipsi vs. CFA-ipsi) for mean co-occurrence: t={t_stat:.3f}, p={p_val:.3f}")

# Example: ANOVA for mean cross-correlation across all ipsi models
data_anova = []
for model, sides in summary_results_corr.items():
    if 'ipsi' in sides:
        for value in sides['ipsi']:
            data_anova.append({'value': value, 'model': model})
df_anova = pd.DataFrame(data_anova)

if not df_anova.empty:
    model_anova = ols('value ~ C(model)', data=df_anova).fit()
    anova_table = sa.stats.anova_lm(model_anova)
    print("\nANOVA for mean cross-correlation (ipsi):")
    print(anova_table)

    # Post-hoc test (Tukey's HSD) if ANOVA is significant
    if anova_table['PR(>F)'][0] < 0.05:
        m_comp = pairwise_tukeyhsd(endog=df_anova['value'], groups=df_anova['model'], alpha=0.05)
        print("\nTukey's HSD Post-hoc Test:")
        print(m_comp)